In [ ]:
pip install lightgbm

In [ ]:
pip install imblearn

In [ ]:
pip install catboost

In [ ]:
pip install optuna

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
import lightgbm as lgb
import optuna
from sklearn.preprocessing import LabelEncoder, StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from sklearn.preprocessing import PolynomialFeatures
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score

from xgboost.callback import EarlyStopping
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
import xgboost as xgb
from itertools import combinations
from copy import deepcopy
from functools import partial
from optuna.integration import OptunaSearchCV
from optuna.samplers import RandomSampler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_selector
from geopy.distance import geodesic
from sklearn.cluster import KMeans

In [ ]:
train = pd.read_csv("/content/Train (25).csv")
test = pd.read_csv("/content/Test (24).csv")

In [ ]:
y = train['Target']
train = train.drop("Target",1)

<ipython-input-8-3d62a69b02f8>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train = train.drop("Target",1)


In [ ]:
train = pd.get_dummies(train["ID"])
test = pd.get_dummies(test["ID"])

In [ ]:
test.head()

,ID_000001,ID_000002,ID_000003,ID_000004,ID_000005,ID_000006,ID_000007,ID_000008,ID_000009,ID_000010,...,ID_039991,ID_039992,ID_039993,ID_039994,ID_039995,ID_039996,ID_039997,ID_039998,ID_039999,ID_040000
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
clm = train.columns
test = test[clm]

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
eng_stopwords = set(stopwords.words('english'))
import string

In [ ]:
train['num_words'] = train['ID'].apply(lambda x: len(str(x).split()))
test['num_words'] = test['ID'].apply(lambda x: len(str(x).split()))

train['num_unique_words'] = train['ID'].apply(lambda x: len(set(str(x).split())))
test['num_unique_words'] = test['ID'].apply(lambda x: len(set(str(x).split())))

train['num_chars'] = train['ID'].apply(lambda x: len(str(x)))
test['num_chars'] = test['ID'].apply(lambda x: len(str(x)))

train['num_stopwrods'] = train['ID'].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test['num_stopwrods'] = test['ID'].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

train['num_punctuations'] = train['ID'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test['num_punctuations'] = test['ID'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

train['num_words_upper'] = train['ID'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test['num_words_upper'] = test['ID'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Average length of the words in the text ##
train['mean_word_len'] = train['ID'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test['mean_word_len'] = test['ID'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
train['distance'] = 1000*np.sqrt(((train['Lat']-train['Lon'])*111.1)**2+ ((train['Lon']-train['Lon'])*111.1)**2)
test['distance'] = 1000*np.sqrt(((test['Lat']-test['Lon'])*111.1)**2+ ((test['Lon']-test['Lon'])*111.1)**2)

In [ ]:
location_of_interest = (32.0, 48.0)
train['Distance'] = train.apply(lambda row: geodesic((row['Lat'], row['Lon']), location_of_interest).kilometers, axis=1)

In [ ]:
location_of_interest = (32.0, 48.0)
test['Distance'] = test.apply(lambda row: geodesic((row['Lat'], row['Lon']), location_of_interest).kilometers, axis=1)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
train['Cluster'] = kmeans.fit_predict(train[['Lat', 'Lon']])
test['Cluster'] = kmeans.predict(test[['Lat', 'Lon']])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
train['Lat_lon_interaction'] = train['Lat'] * train['Lon']
test['Lat_lon_interaction'] = test['Lat'] * test['Lon']

In [ ]:
train['Feature_mean'] = train.mean(axis=1)
test['Feature_mean'] = test.mean(axis=1)

<ipython-input-16-60fcf0403314>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train['Feature_mean'] = train.mean(axis=1)
<ipython-input-16-60fcf0403314>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test['Feature_mean'] = test.mean(axis=1)


In [ ]:
train['Temperature'] = np.random.randint(low=0, high=50, size=len(train))
test['Temperature'] = np.random.randint(low=0, high=50, size=len(test))

In [ ]:
train['distance_log'] = np.log(train['distance'])
test['distance_log'] = np.log(test['distance'])

In [ ]:
def HaversineDistance(lat1,lon1,lat2,lon2):
	"""
	Returns the distance between two lat-long cordinates in km
	"""
	REarth = 6371
	lat = np.abs(np.array(lat1)-np.array(lat2))*np.pi/180
	lon = np.abs(np.array(lon1)-np.array(lon2))*np.pi/180
	lat1 = np.array(lat1)*np.pi/180
	lat2 = np.array(lat2)*np.pi/180
	a = np.sin(lat/2)*np.sin(lat/2)+np.cos(lat1)*np.cos(lat2)*np.sin(lon/2)*np.sin(lon/2)
	d = 2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
	d = REarth*d
	return d

train["Havdistance"] = (
    HaversineDistance(
        train.loc[:, "Lat"], train.loc[:, "Lon"],
        train.loc[:, "Lon"], train.loc[:, "Lat"]
    )
)
test["Havdistance"] = (
    HaversineDistance(
        test.loc[:, "Lat"], test.loc[:, "Lon"],
        test.loc[:, "Lon"], test.loc[:, "Lat"]
    )
)

In [ ]:
test.shape

(1000, 18)

In [ ]:
y = train["Target"]

In [ ]:
train = train.drop(["ID","Target"],axis=1)
test = test.drop("ID",axis=1)

In [ ]:
oversampler = RandomOverSampler(random_state=42)
under_sampler = RandomUnderSampler(random_state=42)

x_train_resampled, y_train_resampled = oversampler.fit_resample(train, y)
X_train, y_train = under_sampler.fit_resample(x_train_resampled, y_train_resampled)

In [ ]:
X_train.shape

(1000, 17)

In [ ]:
test.shape

(1000, 17)

In [ ]:
scale = MinMaxScaler()
train = scale.fit_transform(train)
test = scale.fit_transform(test)

In [ ]:
train = X_train
test = test
y = y_train

In [ ]:
y.unique()

array([1, 0])

In [ ]:
def objective(trial):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = trial.suggest_float('eta', 0.01, 0.1, log=True)
    param['max_depth'] = trial.suggest_int('max_depth', 3, 10)
    param['min_child_weight'] = trial.suggest_float('min_child_weight', 0.1, 10.0, log=True)
    param['subsample'] = trial.suggest_float('subsample', 0.1, 1.0)
    param['colsample_bytree'] = trial.suggest_float('colsample_bytree', 0.1, 1.0)
    param['seed'] = 0
    param['num_class'] = 2
    param['eval_metric'] = 'mlogloss'

    X_train, X_val, y_train, y_val = train_test_split(train, y, test_size=0.1, random_state=42)
    xgtrain = xgb.DMatrix(X_train, label=y_train)
    xgval = xgb.DMatrix(X_val, label=y_val)

    evals_result = {}

    model = xgb.train(param, xgtrain, 3, [(xgtrain, 'train'), (xgval, 'val')],
                      early_stopping_rounds=50, verbose_eval=20, evals_result=evals_result)

    return evals_result['val']['mlogloss'][-1]

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Check if any trials have been completed
if len(study.trials) == 0:
    print("No trials have been completed.")
    exit()

# Get the best trial and its parameters
best_trial = study.best_trial
best_params = best_trial.params
print("Best params:", best_params)

[I 2023-10-02 23:38:58,827] A new study created in memory with name: no-name-057ac9f4-0b0d-4c01-a5a0-fbe42938cb17
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	train-mlogloss:0.65720	val-mlogloss:0.65544
[2]	train-mlogloss:0.59929	val-mlogloss:0.59436


[I 2023-10-02 23:39:33,720] Trial 0 finished with value: 0.5943579993393315 and parameters: {'eta': 0.07101588889671277, 'max_depth': 10, 'min_child_weight': 1.0914584396861469, 'subsample': 0.5440650509836134, 'colsample_bytree': 0.3205496799476214}. Best is trial 0 with value: 0.5943579993393315.


[0]	train-mlogloss:0.65355	val-mlogloss:0.65163
[2]	train-mlogloss:0.59108	val-mlogloss:0.58574


[I 2023-10-02 23:40:04,208] Trial 1 finished with value: 0.5857415720206706 and parameters: {'eta': 0.07830362092819604, 'max_depth': 3, 'min_child_weight': 0.40651128232754, 'subsample': 0.7039530065985028, 'colsample_bytree': 0.5531213147330518}. Best is trial 1 with value: 0.5857415720206706.


...

[0]	train-mlogloss:0.64689	val-mlogloss:0.64466
[2]	train-mlogloss:0.57854	val-mlogloss:0.57256


[I 2023-10-03 00:27:34,959] Trial 98 finished with value: 0.5725621604644486 and parameters: {'eta': 0.08934655662327305, 'max_depth': 7, 'min_child_weight': 0.1798373575729972, 'subsample': 0.1405102544484241, 'colsample_bytree': 0.34481322007157017}. Best is trial 66 with value: 0.5607253195720999.


[0]	train-mlogloss:0.64411	val-mlogloss:0.64175
[2]	train-mlogloss:0.57257	val-mlogloss:0.56626


[I 2023-10-03 00:28:04,418] Trial 99 finished with value: 0.5662636408794064 and parameters: {'eta': 0.09489365300451283, 'max_depth': 8, 'min_child_weight': 0.13431947416905748, 'subsample': 0.11289483418547577, 'colsample_bytree': 0.3601134436066975}. Best is trial 66 with value: 0.5607253195720999.


Best params: {'eta': 0.0999354304938303, 'max_depth': 6, 'min_child_weight': 0.21645208715401726, 'subsample': 0.10004071670955725, 'colsample_bytree': 0.2722965538626651}


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train, y, test_size=0.1, random_state=42)

In [ ]:
xgtrain = xgb.DMatrix(X_train, label=y_train)
xgval = xgb.DMatrix(X_val, label=y_val)

In [ ]:
evals_result = {}

model = xgb.train(best_params, xgtrain,10, [(xgtrain, 'train'), (xgval, 'val')],
                  early_stopping_rounds=50, verbose_eval=20, evals_result=evals_result)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	train-rmse:0.47248	val-rmse:0.47110
[9]	train-rmse:0.36550	val-rmse:0.35472


In [ ]:
test = xgb.DMatrix(test)

In [ ]:
test = model.predict(test)

In [ ]:
test

array([0.7396328, 0.7396328, 0.7396328, ..., 0.7396328, 0.7396328,
       0.7396328], dtype=float32)

In [ ]:
submission = pd.DataFrame({'ID': testsub["ID"], 'Target': test})
submission.to_csv('double.csv', index=False)